In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import os

In [3]:
import mdtraj as md
import parmed as pmd

In [4]:
# OpenMM Imports
import simtk.openmm as mm
import simtk.openmm.app as app
import simtk.unit as u

In [5]:
from parmed import load_file, unit as u
from parmed.openmm import StateDataReporter, NetCDFReporter

In [6]:
from paprika import align, restraints, simulate, utils

2017-11-15 02:47:16 PM OpenMM support: Yes


# Manual OpenMM run through...

## System setup (standard OpenMM code)

In [32]:
print('Creating OpenMM System')
cb6_but = load_file('../test/cb6-but/vac.topo', '../test/cb6-but/vac.crds')
system = cb6_but.createSystem(nonbondedMethod=app.NoCutoff,
                               constraints=app.HBonds, implicitSolvent=app.GBn2,
                               implicitSolventSaltConc=0.1*u.moles/u.liter,
)

# Create the integrator to do Langevin dynamics
integrator = mm.LangevinIntegrator(
                        300*u.kelvin,       # Temperature of heat bath
                        1.0/u.picoseconds,  # Friction coefficient
                        2.0*u.femtoseconds, # Time step
)

# Define the platform to use; CUDA, OpenCL, CPU, or Reference. Or do not specify
# the platform to use the default (fastest) platform
platform = mm.Platform.getPlatformByName('CUDA')
prop = dict(CudaPrecision='mixed') # Use mixed single/double precision

# Create the Simulation object
sim = app.Simulation(cb6_but.topology, system, integrator, platform, prop)

# Set the particle positions
sim.context.setPositions(cb6_but.positions)

Creating OpenMM System


## Minimize

In [15]:
# Minimize the energy
print('Minimizing energy')
sim.minimizeEnergy(maxIterations=500)

Minimizing energy


In [16]:
st = sim.context.getState(getPositions=True,getEnergy=True)
print(st.getPotentialEnergy())

-3489.7370789646575 kJ/mol


In [17]:
sim.reporters.append(app.PDBReporter('minimized_structure.pdb', 1))
sim.step(1)

In [24]:
starting = md.load('vac.pdb')
final = md.load('minimized_structure.pdb')
print('RMSD = {0:1.5f} A'.format(md.rmsd(starting, final)[0]))

RMSD = 0.00492 A


# OpenMM with single atom restratints (from pAPRika)

In [139]:
rest1 = restraints.DAT_restraint()
rest1.continuous_apr = True
rest1.structure_file = 'vac.pdb'
rest1.mask1 = ':CB6@O'
rest1.mask2 = ':BUT@C3'
rest1.attach['target'] = 3.0
rest1.attach['fraction_list'] = [0.00, 0.04, 0.181, 0.496, 1.000]
rest1.attach['fc_final'] = 5.0
rest1.pull['fc'] = rest1.attach['fc_final']
rest1.pull['target_initial'] = rest1.attach['target']
rest1.pull['target_final'] = 10.0
rest1.pull['num_windows'] = 11
rest1.release['target'] = rest1.pull['target_final']
rest1.release['num_windows'] = len(rest1.attach['fraction_list'])
rest1.release['fc_initial'] = rest1.attach['fc_final']
rest1.release['fc_final'] = rest1.attach['fc_final']
rest1.initialize()

2017-11-15 05:16:16 PM Calculating attach targets and force constants...
2017-11-15 05:16:16 PM Method #3
2017-11-15 05:16:16 PM Calculating pull targets and force constants...
2017-11-15 05:16:16 PM Method #1
2017-11-15 05:16:16 PM Calculating release targets and force constants...
2017-11-15 05:16:16 PM Method #1
2017-11-15 05:16:16 PM Number of attach windows = 5
2017-11-15 05:16:16 PM Number of pull windows = 11
2017-11-15 05:16:16 PM Number of release windows = 5
2017-11-15 05:16:16 PM Assigning atom indices ...
2017-11-15 05:16:16 PM Loaded vac.pdb...
2017-11-15 05:16:16 PM There are 1 atoms in the mask :CB6@O  ...
2017-11-15 05:16:16 PM Loaded vac.pdb...
2017-11-15 05:16:16 PM There are 1 atoms in the mask :BUT@C3  ...


In [140]:
print('Creating OpenMM System')
cb6_but = load_file('../test/cb6-but/vac.topo', '../test/cb6-but/vac.crds')
system = cb6_but.createSystem(nonbondedMethod=app.NoCutoff,
                               constraints=app.HBonds, implicitSolvent=app.GBn2,
                               implicitSolventSaltConc=0.1*u.moles/u.liter,
)

# Create the integrator to do Langevin dynamics
integrator = mm.LangevinIntegrator(
                        300*u.kelvin,       # Temperature of heat bath
                        1.0/u.picoseconds,  # Friction coefficient
                        2.0*u.femtoseconds, # Time step
)

# Define the platform to use; CUDA, OpenCL, CPU, or Reference. Or do not specify
# the platform to use the default (fastest) platform
platform = mm.Platform.getPlatformByName('CUDA')
prop = dict(CudaPrecision='mixed') 

Creating OpenMM System


In [141]:
phase = 'pull'
window = 10

print('Trying to move {} and {} to {} A with force constant {} kcal/mol/A**2'.format(
    rest1.index1[0],
    rest1.index2[0],
    rest1.phase[phase]['targets'][window],
    rest1.phase[phase]['force_constants'][window]))

bond_restraint = mm.CustomBondForce('k * (r - r_0)^2')
bond_restraint.addPerBondParameter('k')
bond_restraint.addPerBondParameter('r_0')

r_0 = rest1.phase[phase]['targets'][window] * 0.1 * u.nanometers
k  = rest1.phase[phase]['force_constants'][window] / 0.239 / 0.01 * u.kilojoules_per_mole/u.nanometers**2

print(r0, k)

bond_restraint.addBond(rest1.index1[0], rest1.index2[0],
                       [r_0, k])

system.addForce(bond_restraint)

# Create the Simulation object
sim = app.Simulation(cb6_but.topology, system, integrator, platform, prop)

# Set the particle positions
sim.context.setPositions(cb6_but.positions)

Trying to move 13 and 119 to 10.0 A with force constant 5.0 kcal/mol/A**2
10.0 A 2092.050209205021 kJ/(nm**2 mol)


In [142]:
# Minimize the energy
print('Minimizing energy')
sim.minimizeEnergy(maxIterations=500)

Minimizing energy


In [143]:
positions = sim.context.getState(getPositions=True).getPositions()
app.PDBFile.writeFile(sim.topology, positions, open('minimized_with_restraint.pdb', 'w'))

# OpenMM with a group restratint (from pAPRika)

In [69]:
rest1 = restraints.DAT_restraint()
rest1.continuous_apr = True
rest1.structure_file = 'vac.pdb'
rest1.mask1 = ':CB6@O'
rest1.mask2 = ':BUT@C*'
rest1.attach['target'] = 3.0
rest1.attach['fraction_list'] = [0.00, 0.04, 0.181, 0.496, 1.000]
rest1.attach['fc_final'] = 5.0
rest1.pull['fc'] = rest1.attach['fc_final']
rest1.pull['target_initial'] = rest1.attach['target']
rest1.pull['target_final'] = 10.0
rest1.pull['num_windows'] = 11
rest1.release['target'] = rest1.pull['target_final']
rest1.release['num_windows'] = len(rest1.attach['fraction_list'])
rest1.release['fc_initial'] = rest1.attach['fc_final']
rest1.release['fc_final'] = rest1.attach['fc_final']
rest1.initialize()

2017-11-15 04:39:24 PM Calculating attach targets and force constants...
2017-11-15 04:39:24 PM Method #3
2017-11-15 04:39:24 PM Calculating pull targets and force constants...
2017-11-15 04:39:24 PM Method #1
2017-11-15 04:39:24 PM Calculating release targets and force constants...
2017-11-15 04:39:24 PM Method #1
2017-11-15 04:39:24 PM Number of attach windows = 5
2017-11-15 04:39:24 PM Number of pull windows = 11
2017-11-15 04:39:24 PM Number of release windows = 5
2017-11-15 04:39:24 PM Assigning atom indices ...
2017-11-15 04:39:24 PM Loaded vac.pdb...
2017-11-15 04:39:24 PM There are 1 atoms in the mask :CB6@O  ...
2017-11-15 04:39:24 PM Loaded vac.pdb...
2017-11-15 04:39:24 PM There are 4 atoms in the mask :BUT@C*  ...


In [70]:
print('Creating OpenMM System')
cb6_but = load_file('../test/cb6-but/vac.topo', '../test/cb6-but/vac.crds')
system = cb6_but.createSystem(nonbondedMethod=app.NoCutoff,
                               constraints=app.HBonds, implicitSolvent=app.GBn2,
                               implicitSolventSaltConc=0.1*u.moles/u.liter,
)

integrator = mm.LangevinIntegrator(
                        300*u.kelvin,       # Temperature of heat bath
                        1.0/u.picoseconds,  # Friction coefficient
                        2.0*u.femtoseconds, # Time step
)

platform = mm.Platform.getPlatformByName('CUDA')
prop = dict(CudaPrecision='mixed') 

Creating OpenMM System


In [71]:
phase = 'pull'
window = 10

print('Trying to move {} and {} to {} A with force constant {} kcal/mol/A**2'.format(
    rest1.index1[0],
    rest1.index2,
    rest1.phase[phase]['targets'][window],
    rest1.phase[phase]['force_constants'][window]))

Trying to move 13 and [109, 113, 115, 119] to 10.0 A with force constant 5.0 kcal/mol/A**2


In [72]:
bond_restraint = mm.CustomCentroidBondForce(2, 'k * (distance(g1, g2) - r_0)^2')
bond_restraint.addPerBondParameter('k')
bond_restraint.addPerBondParameter('r_0')

r_0 = rest1.phase[phase]['targets'][window] * u.angstroms
k  = rest1.phase[phase]['force_constants'][window] * u.kilocalories_per_mole/u.angstroms**2

print(r0, k)

g1 = bond_restraint.addGroup(rest1.index1)
g2 = bond_restraint.addGroup(rest1.index2)

print(rest1.index1, rest1.index2)

print(g1, g2)

bond_restraint.addBond([g1, g2],
                       [r_0, k])

system.addForce(bond_restraint)

# Create the Simulation object
sim = app.Simulation(cb6_but.topology, system, integrator, platform, prop)

# Set the particle positions
sim.context.setPositions(cb6_but.positions)

10.0 A 5.0 kcal/(A**2 mol)
[13] [109, 113, 115, 119]
0 1


In [73]:
# Minimize the energy
print('Minimizing energy')
sim.minimizeEnergy(maxIterations=500)

Minimizing energy


In [74]:
st = sim.context.getState(getPositions=True,getEnergy=True)
print(st.getPotentialEnergy())

4372767.572448878 kJ/mol


In [75]:
# sim.reporters.append(app.PDBReporter('minimized_with_com_restraint.pdb', 1))
sim.reporters.append(app.DCDReporter('tmp.dcd', 1))
sim.step(10000)

In [76]:
starting = md.load('vac.pdb')
final = md.load('minimized_with_com_restraint.pdb')
print('RMSD = {0:1.5f} A'.format(md.rmsd(starting, final)[0]))

ValueError: PDB Error: All MODELs must contain the same number of ATOMs